In [33]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
#Sacamos la url
url_oscar = "https://es.wikipedia.org/wiki/Premios_Óscar"
#hacemos la llamada
respuesta_oscar = requests.get(url_oscar)
#imprimimos la respuesta
print("La respuesta de la petición es:", respuesta_oscar.status_code)

La respuesta de la petición es: 200


In [3]:
#Igualamos a la clase BeautifulSoup
sopa_oscar = BeautifulSoup(respuesta_oscar.content, 'html.parser')
print(sopa_oscar.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Premios Óscar - Wikipedia, la enciclopedia libre
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-dis

In [4]:
#Extraemos todas las tablas de la pagina 
tablas = sopa_oscar.findAll('table')
tablas

[<table class="infobox" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;"><tbody><tr><th class="cabecera" colspan="3" style="text-align:center;background-color:#FFEC80;color:inherit;">Premios Óscar</th></tr><tr><td colspan="3" style="text-align:center;background-color:#FFEC80;;">
 <span typeof="mw:File"><a class="mw-file-description" href="/wiki/Archivo:Current_event_marker.svg" title="Evento actual"><img alt="Evento actual" class="mw-file-element" data-file-height="530" data-file-width="764" decoding="async" height="21" src="//upload.wikimedia.org/wikipedia/commons/thumb/5/59/Current_event_marker.svg/30px-Current_event_marker.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/5/59/Current_event_marker.svg/45px-Current_event_marker.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/5/59/Current_event_marker.svg/60px-Current_event_marker.svg.png 2x" width="30"/></a></span> <i><a href="/wiki/95.%C2%AA_edici%C3%B3n_de_los_Premios_%C3%93scar" 

In [5]:
#Obtenemos la tabla que necesitamos ya que hay mas de una tabla en la pagina
nuestra_tabla = tablas[2]

In [6]:
#Conseguimos los encabezados de nuestra tabla. 
lista_encabezados= nuestra_tabla.findAll('th')

In [7]:
#Imrpimimos los encabezados y los hacemos legibles recorriedo la estructura html
encabezados_oscar = [columna.text for columna in lista_encabezados]

In [8]:
#Los encabezados no los necesitamos 
encabezados_oscar_limpio= encabezados_oscar[1:8]
encabezados_oscar_limpio

['Ceremonia\n',
 'Mejor película\n',
 'Mejor director\n',
 'Mejor actor\n',
 'Mejor actriz\n',
 'Mejor actor reparto\n',
 'Mejor actriz reparto\n']

In [9]:
filas = nuestra_tabla.find_all('tr')
#Accedemos a una de las filas 
filas[1]

<tr>
<td align="center"><a class="mw-redirect" href="/wiki/1.%C2%AA_ceremonia_de_los_Premios_%C3%93scar" title="1.ª ceremonia de los Premios Óscar">1°</a>
</td>
<td>16 de mayo<br/>de 1929
</td>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a class="mw-file-description" href="/wiki/Archivo:Flag_of_the_United_States.svg" title="Bandera de Estados Unidos"><img alt="Bandera de Estados Unidos" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Flag_of_the_United_States.svg/20px-Flag_of_the_United_States.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Flag_of_the_United_States.svg/30px-Flag_of_the_United_States.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Flag_of_the_United_States.svg/40px-Flag_of_the_United_States.svg.png 2x" width="20"/></a></span></span> <i><a href="/wiki/Alas_(pel%C3%ADcula)" title="Alas (película)

In [10]:
resultado_oscar = []
#dejamos los encabezados afuera
for fila in filas[1:]:
    #extraemos el texto
    fila_texto = fila.text
    #separamos por salto de linea
    elementos_fila = fila_texto.split("\n")[1:-1]
    resultado_oscar.append(tuple(elementos_fila))
print(resultado_oscar)




[('1°', '', '16 de mayode 1929', '', ' Alas (Wings) (1927)', '', 'Drama:  F. BorzageCom.:  L. Milestone', '', ' E. Jannings(The Last Command)', '', ' J. Gaynor(7th Heaven)', '', '—', '', '—', '', 'Douglas FairbanksWilliam C. deMille', '', ''), ('2°', '', '3 de abrilde 1930', '', ' La melodía de Broadway (The Broadway melody) (1929)', '', ' F. Lloyd(The Divine Lady)', '', ' W. Baxter(In Old Arizona)', '', ' M. Pickford(Coquette)', '', 'William C. deMille', '', ''), ('3°', '', '5 de noviembrede 1930', '', ' Sin novedad en el frente (All quiet on the western front (1930)', '', ' L. MilestoneSin novedad en el frente (All quiet on the western front) (1930)', '', ' G. Arliss(Disraeli)', '', ' N. Shearer(The Divorcee)', '', 'Conrad Nagel', '', ''), ('4°', '', '10 de noviembrede 1931', '', ' Cimarron (1931)', '', ' N. Taurog(Skippy)', '', ' L. Barrymore(A Free Soul)', '', ' M. Dressler(Min and Bill)', '', 'Lawrence Grant', '', ''), ('5°', '', '18 de noviembrede 1932', '', ' Grand Hotel', '', '

In [ ]:
resultado_oscar

re.sub(r'\s*(\r\n|\n|\r)\s*', '', resultado_oscar)

lista_filtrada = [item for item in respuesta_oscar[0] if item not in ['', '—']]

print(lista_filtrada)



In [31]:
lista = ['1°', '', '16 de mayo de 1929', '', 'Alas (Wings) (1927)', '', 'Drama: F. BorzageCom.: L. Milestone', '', 'E. Jannings(The Last Command)', '', 'J. Gaynor(7th Heaven)', '', '—', '', '—', '', 'Douglas FairbanksWilliam C. deMille', '', '']

lista_filtrada = list(filter(lambda x: x not in ['', '—'], lista))

print(lista_filtrada)

['1°', '16 de mayo de 1929', 'Alas (Wings) (1927)', 'Drama: F. BorzageCom.: L. Milestone', 'E. Jannings(The Last Command)', 'J. Gaynor(7th Heaven)', 'Douglas FairbanksWilliam C. deMille']


In [32]:
import re

def filter_data(data):
    return re.sub(r'\s*(\r\n|\n|\r)\s*', '', data)

lista = ['1°', '', '16 de mayo de 1929', '', 'Alas (Wings) (1927)', '', 'Drama: F. BorzageCom.: L. Milestone', '', ' E. Jannings(The Last Command)', '', ' J. Gaynor(7th Heaven)', '', '—', '', '—', '', 'Douglas FairbanksWilliam C. deMille', '', '']

lista_filtrada = [filter_data(item) for item in lista if item not in ['', '—']]

print(lista_filtrada)

['1°', '16 de mayo de 1929', 'Alas (Wings) (1927)', 'Drama: F. BorzageCom.: L. Milestone', ' E. Jannings(The Last Command)', ' J. Gaynor(7th Heaven)', 'Douglas FairbanksWilliam C. deMille']
